In [5]:
import pandas as pd
import motor.motor_tornado

Read CSV File and create connection with mongo DB

In [6]:
mongo_client = motor.motor_tornado.MotorClient('localhost', 27017)
industrial_db = mongo_client.industrial

df = pd.read_csv("./merge_files - Sheet1.csv",dtype=str)

Check if there is any empty reason. If exist fill with Name

In [7]:
for line in df.iloc:
    if str(line["Reason"]) == "nan":
        line["Reason"] = line["Name"]

Parse duration to time delta. This allow to make calculations using pandas

In [8]:
df["Duration"] = pd.to_timedelta(df["Duration"])

Create a dictionary with raw data for each file on CSV

In [9]:
industrial_files:list[dict] = []

list_of_files = df["File"].unique()
for file in list_of_files:
    stops:list[dict] = []
    file_registers = df[ df["File"]==file ]
    for register in file_registers.iterrows():
        stops.append({
            "start":pd.to_datetime(register[1]["Start"]),
            "end":pd.to_datetime(register[1]["End"]),
            "duration":str(register[1]["Duration"]),
            "name":register[1]["Name"],
            "reason":register[1]["Reason"]})

    industrial_files.append({
        "file":file,
        "stops":stops
    })

Save raw data into mongo DB

In [14]:
await industrial_db.raw_data.insert_many(industrial_files)

Process dataframe to generate data

In [11]:
list_processed_files:list[dict] = []

list_of_files = df["File"].unique()
for file in list_of_files:
    data: dict = {}
    data["file"] = file
    data["result"] = []

    file_data:pd.DataFrame = df[ df["File"]==file ]
    total_of_duration = file_data["Duration"].sum()
    list_of_reasons = file_data["Reason"].unique()

    for reason in list_of_reasons:
        duration_total_of_reason = (file_data[file_data["Reason"]==reason])["Duration"].sum()
        percent_of_total = (duration_total_of_reason*100)/total_of_duration

        data["result"].append({
            "reason":reason,
            "duration_total":str(duration_total_of_reason),
            "relative_duration":percent_of_total
        })
    list_processed_files.append(data)

Insert processed data into mongo

In [15]:
await industrial_db.processed_data.insert_many(list_processed_files)